## Batch Transformation and Queries using Databricks and Pyspark

In [0]:
dbutils.fs.ls("/FileStore/tables")

Out[3]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1674066475000)]

###### Run this code to mount the S3 bucket to the databricks notebook. Afterwards this code can be commented out/deleted as it only needs to be run once.

In [0]:
from pyspark.sql.functions import *
import urllib

# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a4e65e909bd-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/pinterest_pipeline"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3060891424582214> in <module> 
 27 SOURCE_URL = "s3n://{0}:{1}@{2}" . format ( ACCESS_KEY , ENCODED_SECRET_KEY , AWS_S3_BUCKET ) 
 28 # Mount the drive 
 ---> 29 dbutils . fs . mount ( SOURCE_URL , MOUNT_NAME ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 387 exc . __context__ = None 
 388 exc . __cause__ = None 
 --> 389 raise exc
 390 
 391 return f_with_exception_handling

 ExecutionError : An error occurred while calling o453.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/pinterest_pipeline; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/pinterest_pipeline
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:756)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:776)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:750)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/pinterest_pipeline
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:557)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$2(MetadataManager.scala:925)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:706)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:914)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:565)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:120)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:54)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:53)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:53)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$4.$anonfun$applyOrElse$5(DbfsServerBackend.scala:374)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:196)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:418)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:22)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:

##### Creating the geo_df and cleaning the data

In [0]:
from pyspark.sql.functions import *

file_location = "/mnt/pinterest_pipeline/topics/0a4e65e909bd.geo/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
geo_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)

geo_df = geo_df.withColumn("coordinates", array(col("longitude"), col("latitude")))

geo_df = geo_df.withColumn("timestamp", col("timestamp").cast("timestamp"))

column_structure = ["ind", "country", "coordinates", "timestamp"]
geo_df = geo_df.select(column_structure)


##### Creating the pin_df and cleaning the data

In [0]:
file_location = "/mnt/pinterest_pipeline/topics/0a4e65e909bd.pin/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
pin_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)

pin_df = pin_df.replace("No description available Story format", None)
pin_df = pin_df.replace("null", None)
pin_df = pin_df.replace("User Info Error", None)
pin_df = pin_df.replace("Image src error.", None)
pin_df = pin_df.replace("No Title Data Available", None)
pin_df = pin_df.replace("N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e", None)

pin_df = pin_df.withColumn("follower_count", when(
    col("follower_count").rlike("\\d+k"), #checks if the value matches a pattern of one or more digits plus the letter k
    (regexp_extract(col("follower_count"), "(\\d+)", 1).cast("integer") * 1000) #extracts the integer from the cells containing k and * 1000
).otherwise(col("follower_count").cast("integer"))) #if it doesn't contain a k, it leaves the integer value

pin_df = pin_df.withColumn("save_location", regexp_extract(col("save_location"), "(/data/).*", 0)) #extracts the save location of the column 

pin_df = pin_df.withColumnRenamed("index", "ind")

column_structure = ["ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category"]
pin_df = pin_df.select(column_structure)


##### Creating the user_df and cleaning the data

In [0]:
file_location = "/mnt/pinterest_pipeline/topics/0a4e65e909bd.user/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
user_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)

user_df = user_df.withColumn("user_name", concat(col("first_name"),col("last_name")))
user_df = user_df.drop("first_name", "last_name")
user_df = user_df.withColumn("date_joined", col("date_joined").cast("timestamp"))

column_structure = ["ind", "user_name", "age", "date_joined"]
user_df = user_df.select(column_structure)
# Display Spark dataframe to check its content


##### Functions created to mitigate repeat code.

In [0]:
def join_df(df1, df2):
    '''
    Joins two databases with the ind column as a common key.

    Parameters
    ----------
    df1 :
        a database to be joined
    df2 :
        a database to be joined

    Returns
    -------
    joined_database :
        a database created by joining db1 and db2 via the ind column
    '''
    joined_df = df1.join(df2, df1.ind == df2.ind, "inner")
    return joined_df

def create_post_year(df, column):
    '''
    Reduces the timestamp of a column to just the year

    Parameters
    ----------
    df :
        df in which the column post_year is to be ammended
    column : str
        the column that is to be changed to post_year
    
    Returns
    -------
    df :
        dataframe with the column showing the post year as an integer
    '''
    df = df.withColumn(f"{column}", df[column].substr(1,4))
    df = df.withColumnRenamed(f"{column}", "post_year")
    df = df.withColumn("post_year", col("post_year").cast("Integer"))
    return df

def create_age_groups(df):
    '''
    Creates age_group column from the age column

    Parameters
    ----------
    df :
        dataframe for the age column to be turned into age_group

    Returns
    -------
    df :
        dataframe with the age_group column added
    '''
    df = df.withColumn("age_group", when((df.age >= 18) & (df.age <= 24), "18-24")
                                             .when((df.age >= 25) & (df.age <= 35), "25-35")
                                             .when((df.age >=36) & (df.age <= 50), "36-50")
                                             .when(df.age > 50, "50+")
                                             .otherwise("Unknown"))
    return df

def find_most_popular_category(df, column):
    '''
    Finds the most popular category in the dataframe given its relation to another column

    Parameters
    ----------
    df :
        dataframe to finds the most popular category from
    column : str
        column to compare the most popular category by

    Returns
    -------
    df :
        dataframe that shows the most popular category count
    '''
    df = df.groupBy(column, "category").count().withColumnRenamed("count", "category_count")
    sorted_df = df.orderBy(col("category_count").desc())
    df = sorted_df.groupBy(column).agg( #sorting by the column
    first("category").alias("most_popular_category"), #gets the data frame from the first/most popular category
    first("category_count").alias("category_count"))
    return df

Most popular category in each country

In [0]:
category_country_df = join_df(pin_df, geo_df)
most_popular_category_df = find_most_popular_category(category_country_df, "country")

display(most_popular_category_df)

country,most_popular_category,category_count
Albania,mens-fashion,50
Anguilla,home-decor,1
Armenia,diy-and-crafts,40
Aruba,tattoos,6
Azerbaijan,event-planning,1
Bulgaria,finance,1
Cambodia,diy-and-crafts,1
Cocos (Keeling) Islands,vehicles,4
Colombia,finance,12
Cote d'Ivoire,education,4


Most popular category each year

In [0]:
post_year_geo_df = create_post_year(geo_df, "timestamp")
post_year_category_df = join_df(post_year_geo_df, pin_df)
post_year_category_df = find_most_popular_category(post_year_category_df, "post_year")
post_year_category_filtered_df = post_year_category_df.filter((col("post_year") >= 2018) & (col("post_year")<= 2022))
most_popualr_category_by_year_df = post_year_category_filtered_df.orderBy(col("post_year").desc())

display(most_popualr_category_by_year_df)

post_year,most_popular_category,category_count
2022,vehicles,4
2021,finance,13
2020,mens-fashion,50
2019,christmas,49
2018,beauty,6


User with most followers in each country

In [0]:
country_poster_name_follower_df = join_df(pin_df, geo_df)
country_poster_name_follower_df = country_poster_name_follower_df.dropDuplicates(["country", "poster_name", "follower_count"])
country_poster_name_follower_df = country_poster_name_follower_df.select("country", "poster_name", "follower_count")

country_poster_name_follower_df = country_poster_name_follower_df.orderBy(col("follower_count").desc())
most_followers_by_country_df = country_poster_name_follower_df.groupBy("country").agg(
    first("follower_count").alias("follower_count"),
    first("poster_name").alias("poster_name")
)

display(most_followers_by_country_df)

country,follower_count,poster_name
Albania,null,null
Anguilla,92000,"Kristen | Lifestyle, Mom Tips & Teacher Stuff Blog"
Armenia,124000,Of Life & Lisa | Lifestyle Blog
Aruba,211000,TheTrendSpotter
Azerbaijan,null,Style Me Pretty
Bulgaria,26000,"Living Low Key | Save Money, Make Money, & Frugal Living"
Cambodia,6000,Mixed Media Crafts
Cocos (Keeling) Islands,437,Ray Uyemura
Colombia,0,Consuelo Aguirre
Cote d'Ivoire,192000,The Crafting Chicks


User with most followers overall

In [0]:
most_followers_country_df = country_poster_name_follower_df.orderBy(col("follower_count").desc())
most_followers_country_df = most_followers_country_df.drop("poster_name")
most_followers_overall_df = most_followers_country_df.head(1)

display(most_followers_overall_df)

country,follower_count
Aruba,211000


Most popular category for each age group

In [0]:
joined_pin_user_df = join_df(pin_df, user_df)
category_age_df = joined_pin_user_df.select("category", "age")
category_age_df = create_age_groups(category_age_df)
grouped_category_age_df = category_age_df.groupBy("category", "age_group").count().withColumnRenamed("count", "category_count")
grouped_category_age_df = grouped_category_age_df.orderBy(col("category_count").desc())
most_popular_category_by_age_group_df = grouped_category_age_df.groupBy("age_group").agg(
    first("category").alias("category"),
    first("category_count").alias("category_count")
)

display(most_popular_category_by_age_group_df)

age_group,category,category_count
18-24,mens-fashion,46
25-35,diy-and-crafts,36
36-50,finance,9


Median follower count for each age group

In [0]:
joined_pin_user_df = pin_df.join(user_df, pin_df.ind == user_df.ind, "inner")
follower_age_df = joined_pin_user_df.select("follower_count", "age")
follower_age_df = create_age_groups(follower_age_df)

age_group_median_df = follower_age_df.groupBy("age_group").agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))
age_group_median_df = age_group_median_df.orderBy(col("age_group").asc())

display(age_group_median_df) 

age_group,median_follower_count
18-24,211000
25-35,79000
36-50,0


Number of users joined each year

In [0]:
users_joined_2015_2020 = create_post_year(user_df, "date_joined")
users_joined_2015_2020 = users_joined_2015_2020.groupBy("post_year").count().withColumnRenamed("count","number_users_joined")
users_joined_2015_2020 = users_joined_2015_2020.orderBy(col("post_year").asc())

display(users_joined_2015_2020)

post_year,number_users_joined
2015,23
2016,12
2017,10


Median follower count for users based on their join year

In [0]:
pin_user_df = join_df(pin_df, user_df)
post_date_follower_df = pin_user_df.select("date_joined", "follower_count")
post_year_follower_df = create_post_year(post_date_follower_df, "date_joined")

year_follower_median_df = post_year_follower_df.groupBy("post_year").agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

display(year_follower_median_df)

post_year,median_follower_count
2015,25000
2016,124000
2017,79000


Median follower count based on age group and join year

In [0]:
pin_user_df = join_df(pin_df, user_df)
follower_age_date_df = pin_user_df.select("follower_count", "age", "date_joined")
follower_age_date_df = create_age_groups(follower_age_date_df)
follower_age_year_df = follower_age_date_df.drop("age")
follower_age_year_df = create_post_year(follower_age_date_df, "date_joined")

follower_age_year_median_df = follower_age_year_df.groupBy("age_group", "post_year").agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))
follower_age_year_median_df = follower_age_year_median_df.sort(["age_group", "post_year"], ascending = True)

display(follower_age_year_median_df)


age_group,post_year,median_follower_count
18-24,2015,211000
18-24,2017,940
25-35,2015,51000
25-35,2016,124000
25-35,2017,79000
36-50,2015,25000
36-50,2017,6000
